# Structured Outputs
This notebook demonstrates how to work with different output formats and parsers when working with Language Models (LLM).

## What we'll learn:
- Basic string output parsing
- Working with tools and structured outputs
- Using Pydantic models for type validation
- Understanding different parser types and their use cases

### Setup

In [1]:
from typing import Annotated
from pydantic import BaseModel, Field
from dotenv import load_dotenv
from lib.messages import UserMessage, SystemMessage
from lib.tooling import tool
from lib.llm import LLM
from lib.parsers import (
    StrOutputParser,
    JsonOutputParser, 
    PydanticOutputParser, 
    ToolOutputParser,
)

In [2]:
load_dotenv()

True

In [3]:
chat_model = LLM()

## Basic String Output
Before diving into more complex output formats, let's understand how to work with simple string outputs from our Language Model.
This demonstrates the most basic form of parsing LLM responses.

In [4]:
messages = [
    SystemMessage(content="Extract the event information."),
    UserMessage(content="Alice and Bob are going to a science fair on Friday.")
]

In [5]:
ai_message = chat_model.invoke(messages)
ai_message

AIMessage(content='Event: Science Fair  \nParticipants: Alice and Bob  \nDay: Friday', role='assistant', tool_calls=None)

In [6]:
parser = StrOutputParser()
print(parser.parse(ai_message))

Event: Science Fair  
Participants: Alice and Bob  
Day: Friday


## Working with Tools
Now let's see how we can use tools to get structured outputs from our LLM.
Tools help us enforce a specific format for the output and make it easier to process programmatically.

In [7]:
@tool
def calendar_event(name:str, date:str, participants:list[str]):
    """Identify name of the event, date when it will happen and all the participants"""
    return {
        "name": name,
        "date": date,
        "participants": participants
    }

In [8]:
messages = [
    SystemMessage(content="Extract the event information."),
    UserMessage(content="Alice and Bob are going to a science fair on Friday.")
]

In [9]:
chat_model_with_tools = LLM(tools=[calendar_event])
ai_message = chat_model_with_tools.invoke(messages)

In [10]:
# This gives us the variables identified by the LLM
parser = ToolOutputParser()
parser.parse(ai_message)[0]["args"]

{'name': 'Science Fair', 'date': 'Friday', 'participants': ['Alice', 'Bob']}

## Using Pydantic Models
Pydantic provides a powerful way to validate and structure our LLM outputs.
This approach gives us type safety and data validation out of the box.

In [11]:
class CalendarEvent(BaseModel):
    """A Pydantic model representing a calendar event.
    
    This model defines the structure for calendar events with validation:
    - name: The name/title of the event
    - date: When the event will occur
    - participants: List of people attending the event
    """

    name: Annotated[str, Field(description="Name/Title of the event. Defaults to ''", default=None)]
    date: Annotated[str, Field(description="Date of the event. Defaults to ''", default=None)]
    participants: Annotated[list[str], Field(description="Who will participate. Defaults to ''", default=None)]

In [12]:
messages = [
    SystemMessage(content="Extract the event information."),
    UserMessage(content="Alice and Bob are going to a science fair on Friday.")
]

In [13]:
ai_message = chat_model.invoke(input=messages, response_format=CalendarEvent)
ai_message

AIMessage(content='{"name":"Science Fair","date":"Friday","participants":["Alice","Bob"]}', role='assistant', tool_calls=None)

In [14]:
# This gives us a structured dictionary we can work with programmatically
parser = JsonOutputParser()
parser.parse(ai_message)

{'name': 'Science Fair', 'date': 'Friday', 'participants': ['Alice', 'Bob']}

In [15]:
# This gives us a validated Pydantic model with type checking
parser = PydanticOutputParser(model_class=CalendarEvent)
event:CalendarEvent = parser.parse(ai_message)
event

CalendarEvent(name='Science Fair', date='Friday', participants=['Alice', 'Bob'])

In [16]:
event.participants

['Alice', 'Bob']